In [ ]:
import os
import talib
import logging
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
root_path = "/Users/fang/stock_data/basic_data"
# root_path = "C:\\Users\\Fang\\PycharmProjects\\股票分析\\stock_data\\basic_data"
# root_path = "C:\\Users\\Flora\\Documents\\股票分析\\stock_data\\basic_data"

In [ ]:
df_list = []
now = datetime.now()
start_dt = (now-timedelta(days=60)).strftime('%Y-%m-%d')
end_dt = now.strftime('%Y-%m-%d')
bband_slope_change_criteria = 0.5
bband_width_change_criteria = 0.25
prev_bband_width_criteria = 5
for file in sorted(os.listdir(root_path)):
    if '.csv' not in file:
        continue
    logging.info(file.replace('.csv', ''))
    
    # load stock price data
    file_path = root_path + os.sep + file
    try:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python', encoding='utf-8').sort_values('日期')
    except:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python').sort_values('日期')
    stock_df.loc[:, '日期'] = pd.to_datetime(stock_df['日期'].str.replace('＊', ''))
    stock_df = stock_df[(stock_df['日期'] >= start_dt) & (stock_df['日期'] <= end_dt)]
    if len(stock_df) < 40:
        continue
    stock_df = stock_df.sort_values('日期')
    stock_df['收盤價'].fillna(method='ffill', inplace=True)
    
    # calculate BBands and some features
    closed = stock_df['收盤價'].values
    upper,middle,lower = talib.BBANDS(closed,20, 2, 2, matype=talib.MA_Type.SMA)
    previous_upper = pd.Series(upper).shift(periods=1).to_numpy()
    previous_lower = pd.Series(lower).shift(periods=1).to_numpy()
    upper_change = pd.Series((upper - previous_upper) / previous_upper*100)
    lower_change = (lower - previous_lower) / previous_lower*100
    bband_width =  pd.Series((upper-lower) / middle*100)
    bband_width_ma = pd.Series(talib.SMA(bband_width, 5))
    
    # add BBands data to stock price dataframe
    stock_df = stock_df.reset_index().drop('index', axis=1)
    stock_df['隔日最高價'] = stock_df['最高價'].shift(periods=-1)
    stock_df['隔日收盤價'] = stock_df['收盤價'].shift(periods=-1)
    stock_df['bband_width'] = bband_width
    stock_df['prev_bband_width'] = stock_df['bband_width'].shift(periods=1)
    stock_df['bband_slope'] = upper_change
    stock_df['prev_bband_slope'] = stock_df['bband_slope'].shift(periods=1)
    stock_df['bband_slope_change'] = abs(stock_df['bband_slope'] - stock_df['prev_bband_slope']) / (abs(stock_df['prev_bband_slope'] + 0.00000001))
    stock_df['bband_width_change'] = (stock_df['bband_width'] - stock_df['prev_bband_width']) / (stock_df['prev_bband_width'] + 0.00000001)
    
    # filter data if meet the alert conditions
    filtered_df = stock_df[(stock_df['bband_slope_change']>bband_slope_change_criteria) 
                           & (stock_df['bband_width_change']>bband_width_change_criteria) 
                           & (stock_df['prev_bband_width']<prev_bband_width_criteria)]
    df_list.append(filtered_df)
alert_df = pd.concat(df_list, axis=0, ignore_index=True, sort=False)

In [ ]:
alert_df[alert_df['日期']=='2020-07-23'].sort_values(['日期', '成交金額'])